In [1]:
import numpy as np
import pandas as pd

In [2]:
testpath = '/Users/andrescrucettanieto/Documents/GitHub/advent_of_code/2021/data/09_test.txt'
fullpath = '/Users/andrescrucettanieto/Documents/GitHub/advent_of_code/2021/data/09.txt'

def read_input(filepath):
    '''
    Reads matrix from the filepath.
    '''
    with open(filepath) as f:
        data = f.readlines()
    rows = list(map(lambda x: x.strip('\n'),data))
    matrix = []
    for row in rows:
        split_row = [int(r) for r in row]
        matrix.append(split_row)
    return np.array(matrix)

In [3]:
test_matrix = read_input(testpath)
full_matrix = read_input(fullpath)

In [4]:
full_matrix

array([[5, 4, 5, ..., 8, 9, 9],
       [4, 3, 4, ..., 6, 6, 8],
       [1, 2, 9, ..., 4, 5, 7],
       ...,
       [1, 0, 1, ..., 9, 6, 7],
       [2, 1, 2, ..., 4, 5, 6],
       [3, 6, 5, ..., 2, 3, 7]])

### Part 1

General approach:
- Iterate over each cell
- Get the values around each of the cells
- Find cell indices where all of the values adjacent to each cell are lower than the cell
- Add the risk level to the low points and then sum them

In [5]:
def get_valleys(arr):
    '''
    Returns a list of the values of each
    of the valleys in an nxp array.
    '''
    rows, columns = np.shape(arr)
    rows = rows-1 
    columns = columns-1
    lowest = []
    for i,row in enumerate(arr):
        for j,cel in enumerate(row):
            curr = cel
            smallest = True
            # Checking for adjacent conditions
            left, right = True, True
            down, up = True, True
            
            if (i-1)<0: up = False
            if (i+1)>rows: down = False
            if (j-1)<0: left = False
            if (j+1)>columns: right = False
            
            if left:
                left = arr[i,j-1]
                if left <= curr: 
                    smallest = False
            if right:
                right = arr[i,j+1]
                if right <= curr: 
                    smallest = False
            if up:
                up = arr[i-1,j]
                if up <= curr: 
                    smallest = False
            if down:
                down = arr[i+1,j]
                if down <= curr:
                    smallest = False
            if smallest:
                lowest.append(curr)
    return lowest

In [6]:
def calculate_risk(valleys):
    '''
    Calculates the risk levels of all low points in the height map
    '''
    sum = 0
    for i in valleys:
        sum += i+1
    return sum

In [7]:
calculate_risk(get_valleys(full_matrix))

506

### Part 2

In [8]:
test_matrix

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

General approach:
- Make 9's values -1
- Do a search starting from the lowest number around and find all of the connected values

In [12]:
test_matrix_neg = np.where(test_matrix==9,-1,test_matrix)

In [57]:
full_matrix

array([[5, 4, 5, ..., 8, 9, 9],
       [4, 3, 4, ..., 6, 6, 8],
       [1, 2, 9, ..., 4, 5, 7],
       ...,
       [1, 0, 1, ..., 9, 6, 7],
       [2, 1, 2, ..., 4, 5, 6],
       [3, 6, 5, ..., 2, 3, 7]])

In [61]:
from collections import deque as queue

dRow = [-1,0,1,0]
dCol = [0,1,0,-1]
full_matrix_neg = np.where(full_matrix==9,-1,full_matrix)

# Function to check if a cell is in bounds
def isValid(grid,vis,row,col):
    
    # If cell is out of bounds, return False
    if (row < 0 or col < 0 or row >= len(vis) or col >= len(vis[0])):
        return False
    
    # Check if cell is -1
    if grid[row][col] == -1:
        return False
    
    # If cell is already visited, return False
    if (vis[row][col]):
        return False
    
    # If cell is valid, return True
    return True
    
# Function to perform the BFS traversal for the numpy grid
def BFS(grid, vis, row, col):
    
    # Store index of the queue
    q = queue()
    
    # Marked the current cell as visited
    # and enqueue it
    q.append([row,col])
    vis[row][col] = True
    
    # Storing the value of the current cell
    storage = []
    
    # Iterate while the queue is not empty
    while (len(q)>0):
        cell = q.popleft()
        x = cell[0]
        y = cell[1]
        # print(grid[x][y],end=" ")
        storage.append(grid[x][y])
        
        # Iterate through all adjacent cells
        for i in range(4):
            if isValid(grid,vis,x+dRow[i],y+dCol[i]):
                q.append([x+dRow[i],y+dCol[i]])
                vis[x+dRow[i]][y+dCol[i]] = True
    return storage

In [62]:
# Declare the visited matrix
vis = [[False for i in range(len(test_matrix[0]))] for j in range(len(test_matrix))]

# vis, False for all cells
lst = BFS(test_matrix_neg,vis,0,9)

In [63]:
# Test GFS for all cells
def iterate_BFS(grid):
    vis = [[False for i in range(len(grid[0]))] for j in range(len(grid))]
    iter_lst = []
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] != -1 and not vis[i][j]:
                iter_lst.append([BFS(grid,vis,i,j),i,j])
    return iter_lst
                

In [64]:
def get_top_three_basin(lst):
    '''
    Get the basin with the 3 longest values and
    multiplies their length together
    '''
    length_lst = []
    for nums in lst:
        length = len(nums[0])
        length_lst.append(length)
    # Return the top 3 largest values
    top_3 = sorted(length_lst,reverse=True)[:3]
    
    # Multiply the top 3 values
    return top_3[0]*top_3[1]*top_3[2]
        

In [67]:
lst_basins = iterate_BFS(full_matrix_neg)

In [68]:
get_top_three_basin(lst_basins)

931200